In [2]:
import difflib
from sklearn.metrics import precision_score, recall_score, f1_score
from typing import List, Tuple, Dict, Any
import requests
import json
import pandas as pd
from IPython.display import display
from openai import OpenAI
import json
import difflib
from sklearn.metrics import precision_score, recall_score, f1_score
from typing import List, Tuple, Dict, Any
import requests
import json
import pandas as pd
import json
from IPython.display import display
from openai import OpenAI
from tqdm import tqdm
from tool import soft_match_score,output_tf,filter,merge,add_slang_prompt,assembly_prompt

In [13]:
# Target数量
gold_ref = 0
pred_ref = 0
# 真正例（True Positive，TP） 正确匹配
# 硬匹配
hard_match = 0
hard_match_list = []
# 软匹配
soft_match = 0
soft_match_list = []

mathc_hateError = 0
mathc_hateError_list = []

# 假正例（False Positive，FP）预测中未在黄金标准中出现的
fp_ref = 0
fp_ref_list = []
# 假负例（False Negative，FN） 黄金标准中未在预测中出现的
fn_ref = 0
fn_ref_list = []
LOG = {}
LOG['len_gold'] = {}
LOG['len_pred'] = {}
import random
import time
# 随机种子
random.seed(time.time())
with open('../output3.json', 'r', encoding='utf-8') as file:
    lines = file.readlines()
    for i in range(len(lines)):
        lines[i] = lines[i].strip()
    # 随机访问
    lines = random.sample(lines, len(lines))
    for line in tqdm(lines):
        temp = json.loads(line)



        # if len(temp['content']) < 100:
            # continue

        if '[SEP]' not in temp['output']:
            continue




        res = temp['results'].replace(' ', '').replace('\n', '')
        res = res.replace("'指代':", "'Target':").replace("'评论':", "'Argument':").replace("'仇恨类型':", "'HateType':").replace("'是否仇恨':", "'Is':").replace("'对象':", "'Target':").replace("'是'", "'y'").replace("'否'", "'n'") 
        res = '"' + res + '"'
        try:            
            pred_quads = eval(res)
            pred_quads = eval(pred_quads)
        except:
            print('eval错误',res)
            continue
        gold_quads = output_tf(temp['output'])
        if LOG['len_gold'].get(len(gold_quads)) == None:
            LOG['len_gold'][len(gold_quads)] = 1
        else:
            LOG['len_gold'][len(gold_quads)] += 1
        if LOG['len_pred'].get(len(pred_quads)) == None:
            LOG['len_pred'][len(pred_quads)] = 1
        else:
            LOG['len_pred'][len(pred_quads)] += 1
            

        pred_quads = pred_quads[:2]
        ttt = merge(pred_quads,temp['content'])
        pred_quads = filter(pred_quads = pred_quads) 
        gold_ref += len(gold_quads)
        pred_ref += len(pred_quads)
        for i_pred in pred_quads:
            pred_Target = i_pred['Target']
            pred_Argument = i_pred['Argument']
            pred_Is = i_pred['Is']
            for index_gold in range(len(gold_quads)):
                gold_Target = gold_quads[index_gold]['Target']
                gold_Argument = gold_quads[index_gold]['Argument']
                gold_Is = gold_quads[index_gold]['Is']
                if pred_Target == gold_Target and pred_Argument == gold_Argument:
                    if pred_Is != gold_Is:
                        # if pred_Is == 'y':
                        # print(gold_quads[index_gold])
                        # print(temp['content'])
                        mathc_hateError += 1
                        mathc_hateError_list.append(i_pred)
                    else:
                        hard_match += 1
                        hard_match_list.append(i_pred)
                        gold_quads.pop(index_gold)
                    break
                elif soft_match_score(pred_Target, gold_Target, pred_Argument, gold_Argument):
                    if pred_Is != gold_Is:
                        # if pred_Is == 'y':
                        # print(gold_quads[index_gold])
                        # print(temp['content'])
                        mathc_hateError += 1
                        mathc_hateError_list.append(i_pred)
                    else:
                        soft_match += 1
                        soft_match_list.append(i_pred)
                        gold_quads.pop(index_gold)
                    break
                else:
                    fp_ref += 1
                    fp_ref_list.append(i_pred)
        for i_gold in gold_quads:
            fn_ref += 1
            fn_ref_list.append(i_gold)
        # if len(fp_ref_list) + len(fn_ref_list) > 0:
        #     print('硬匹配和软匹配========================================')
        #     print(hard_match_list)
        #     print(soft_match_list)
        #     print('假正例',fp_ref_list)
        #     print('假负例',fn_ref_list)
        #     print('mathc_hateError',mathc_hateError_list)
        #     print(output_tf(temp['output']))
        #     print(res)
        #     print(temp['content'])
        hard_match_list = []
        soft_match_list = []
        fp_ref_list = []
        fn_ref_list = []
        mathc_hateError_list = []
        

# 计算精确率、召回率和F1值
hard_precision = hard_match / (hard_match + fp_ref) if (hard_match + fp_ref) > 0 else 0
hard_recall = hard_match / (hard_match + fn_ref) if (hard_match + fn_ref) > 0 else 0
hard_f1_score = 2 * hard_precision * hard_recall / (hard_precision + hard_recall) if (hard_precision + hard_recall) > 0 else 0

soft_match_precision = soft_match / (soft_match + fp_ref) if (soft_match + fp_ref) > 0 else 0
soft_match_recall = soft_match / (soft_match + fn_ref) if (soft_match + fn_ref) > 0 else 0
soft_match_f1_score = 2 * soft_match_precision * soft_match_recall / (soft_match_precision + soft_match_recall) if (soft_match_precision + soft_match_recall) > 0 else 0
print('hard_precision:', hard_precision)
print('hard_recall:', hard_recall)
print('hard_f1_score:', hard_f1_score)
print('soft_match_precision:', soft_match_precision)
print('soft_match_recall:', soft_match_recall)
print('soft_match_f1_score:', soft_match_f1_score)
print('**f1**',(hard_f1_score+soft_match_f1_score)/2)

print(f"Hard Match: {hard_match}")
print(f"Soft Match: {soft_match}")
print(f"FP:假正例（False Positive，FP）预测中未在黄金标准中出现的", fp_ref)
print(f"FN:假负例（False Negative，FN） 黄金标准中未在预测中出现的", fn_ref)
print(f"Gold Ref: {gold_ref}")
print(f"Pred Ref: {pred_ref}")
print(f"Mathc HateError: {mathc_hateError}")
print('LOG:',LOG)

100%|██████████| 4000/4000 [00:00<00:00, 57450.12it/s]

hard_precision: 0.1575492341356674
hard_recall: 0.22153846153846155
hard_f1_score: 0.1841432225063939
soft_match_precision: 0.22274562584118437
soft_match_recall: 0.3036697247706422
soft_match_f1_score: 0.25698757763975155
**f1** 0.22056540007307274
Hard Match: 216
Soft Match: 331
FP:假正例（False Positive，FP）预测中未在黄金标准中出现的 1155
FN:假负例（False Negative，FN） 黄金标准中未在预测中出现的 759
Gold Ref: 1306
Pred Ref: 1110
Mathc HateError: 78
LOG: {'len_gold': {2: 506, 3: 90, 4: 6}, 'len_pred': {2: 457, 1: 94, 4: 9, 6: 1, 3: 41}}


In [ ]:
100%|██████████| 4000/4000 [00:00<00:00, 57678.44it/s]
hard_precision: 0.15506547208821503
hard_recall: 0.2348643006263048
hard_f1_score: 0.18679950186799502
soft_match_precision: 0.22109275730622618
soft_match_recall: 0.32192414431082333
soft_match_f1_score: 0.2621468926553673
**f1** 0.22447319726168113
Hard Match: 225
Soft Match: 348
FP:假正例（False Positive，FP）预测中未在黄金标准中出现的 1226
FN:假负例（False Negative，FN） 黄金标准中未在预测中出现的 733
Gold Ref: 1306
Pred Ref: 1173
Mathc HateError: 80
LOG: {'len_gold': {2: 506, 3: 90, 4: 6}, 'len_pred': {2: 457, 3: 41, 1: 94, 4: 9, 6: 1}}

100%|██████████| 4000/4000 [00:00<00:00, 35075.81it/s]
hard_precision: 0.16654330125832717
hard_recall: 0.2348643006263048
hard_f1_score: 0.194889562581204
soft_match_precision: 0.23609226594301222
soft_match_recall: 0.32192414431082333
soft_match_f1_score: 0.27240704500978474
**f1** 0.23364830379549437
Hard Match: 225
Soft Match: 348
FP:假正例（False Positive，FP）预测中未在黄金标准中出现的 1126
FN:假负例（False Negative，FN） 黄金标准中未在预测中出现的 733
Gold Ref: 1306
Pred Ref: 1173
Mathc HateError: 80
LOG: {'len_gold': {2: 506, 3: 90, 4: 6}, 'len_pred': {2: 457, 1: 94, 4: 9, 3: 41, 6: 1}}

100%|██████████| 4000/4000 [00:00<00:00, 13768.94it/s]
hard_precision: 0.23581429624170966
hard_recall: 0.29038112522686027
hard_f1_score: 0.2602684017893453
soft_match_precision: 0.3100465735196274
soft_match_recall: 0.3733974358974359
soft_match_f1_score: 0.3387858960378044
**f1** 0.29952714891357485
Hard Match: 960
Soft Match: 1398
FP:假正例（False Positive，FP）预测中未在黄金标准中出现的 3111
FN:假负例（False Negative，FN） 黄金标准中未在预测中出现的 2346
Gold Ref: 4704
Pred Ref: 5650
Mathc HateError: 526
LOG: {'len_gold': {1: 3398, 2: 506, 3: 90, 4: 6}, 'len_pred': {2: 1447, 1: 2464, 3: 74, 4: 11, 6: 1, 7: 1, 8: 1, 5: 1}}



100%|██████████| 4000/4000 [00:00<00:00, 97566.35it/s]
hard_precision: 0.09818181818181818
hard_recall: 0.1935483870967742
hard_f1_score: 0.13027744270205063
soft_match_precision: 0.14922813036020582
soft_match_recall: 0.27884615384615385
soft_match_f1_score: 0.19441340782122904
**f1** 0.16234542526163984
Hard Match: 54
Soft Match: 87
FP:假正例（False Positive，FP）预测中未在黄金标准中出现的 496
FN:假负例（False Negative，FN） 黄金标准中未在预测中出现的 225
Gold Ref: 366
Pred Ref: 527
Mathc HateError: 17
LOG: {'len_gold': {1: 130, 2: 73, 3: 26, 4: 3}, 'len_pred': {2: 178, 3: 41, 4: 8, 8: 1, 5: 1, 1: 3}}


In [ ]:
c = {'Target':'电脑','Argument':'不会生孩子警告，也不会要这要那还说你坏话','content':'因为电脑不会生孩子警告，也不会要这要那还说你坏话'}
assembly_prompt(content=c,work=2,isSlang=True)

In [ ]:
import jiagu

# 吻别是由张学友演唱的一首歌曲。
# 《盗墓笔记》是2014年欢瑞世纪影视传媒股份有限公司出品的一部网络季播剧，改编自南派三叔所著的同名小说，由郑保瑞和罗永昌联合导演，李易峰、杨洋、唐嫣、刘天佐、张智尧、魏巍等主演。

text = '不是已经入狱了吗，话说回来，我心中反黑领导人是快手发展黑人祝福起家，领导一群肌肉黑人喃小弟的壮硕中国男人。当地黑社会一霸，并且每个黑人工资一个月只有一千块人民币。真正实现了心理生理双重奴役黑人喃'
knowledge = jiagu.knowledge(text)
print(knowledge)